# Figure 4.6 of Chapter 4

In [ ]:
from summer2 import CompartmentalModel
from summer2.parameters import Parameter
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Parameterise
population = 100000.
latent_period = 8.
infectious_period = 7.
r0 = 13.
infectious_seed = 1.

parameters = {
    "contact_rate": r0 / infectious_period,
    "infous_rate": 1. / latent_period,
    "rec_rate": 1. / infectious_period,
}

In [ ]:
# Create equivalent model
compartments = (
    "Susceptible", 
    "Pre-infectious", 
    "Infectious", 
    "Immune"
)
seir_model = CompartmentalModel(
    times=(0., 120.),
    compartments=compartments,
    infectious_compartments=("Infectious",),
)
seir_model.set_initial_population(
    distribution={
        "Susceptible": population - infectious_seed, 
        "Infectious": infectious_seed
    }
)
seir_model.add_infection_frequency_flow(
    name="infection", 
    contact_rate=Parameter("contact_rate"),
    source="Susceptible",
    dest="Pre-infectious"
)
seir_model.add_transition_flow(
    name="progression", 
    fractional_rate=Parameter("infous_rate"),
    source="Pre-infectious", 
    dest="Infectious"
)
seir_model.add_transition_flow(
    name="recovery", 
    fractional_rate=Parameter("rec_rate"), 
    source="Infectious", 
    dest="Immune"
)
seir_model.request_output_for_flow(
    name="incidence", 
    flow_name="progression",
    raw_results=True,
)

In [ ]:
# Run our SEIR model
seir_model.run(parameters=parameters)
seir_compartments = seir_model.get_outputs_df()
outputs = seir_compartments[["Infectious"]]
outputs["New infectious individuals/day"] = seir_model.get_derived_outputs_df()["incidence"]
outputs["Cumulative number of infectious individuals"] = seir_compartments[
    [
        "Pre-infectious", 
        "Infectious", 
        "Immune"
    ]
].sum(axis=1)

In [ ]:
fig, (left_ax, right_ax) = plt.subplots(1, 2, figsize=(14, 5))
log_outputs = np.log(outputs)

for indicator in outputs.columns:
    left_ax.plot(
        seir_model.times,
        log_outputs[indicator],
        label=indicator,
    )
left_ax.set_xlim(0., 120.)
left_ax.set_xlabel("Time (days)")
left_ax.set_ylim(0., 12.)
left_ax.set_ylabel("ln {number or cumulative number of infectious individuals}")
left_ax.set_title("Measles")
left_ax.legend()

image = plt.imread("./figures/fig_4_6_measles.jpg")
right_ax.axis("off")
right_ax.imshow(image, aspect="auto");

In [ ]:
# Re-parameterise for flu
latent_period = 2.
infectious_period = 2.
r0 = 2.
parameters = {
    "contact_rate": r0 / infectious_period,
    "infous_rate": 1. / latent_period,
    "rec_rate": 1. / infectious_period,
}

In [ ]:
# Run our SEIR model
seir_model.run(parameters=parameters)
seir_compartments = seir_model.get_outputs_df()
outputs = seir_compartments[["Infectious"]]
outputs["New infectious individuals/day"] = seir_model.get_derived_outputs_df()["incidence"]
outputs["Cumulative number of infectious individuals"] = seir_compartments[
    [
        "Pre-infectious", 
        "Infectious", 
        "Immune"
    ]
].sum(axis=1)

In [ ]:
fig, (left_ax, right_ax) = plt.subplots(1, 2, figsize=(14, 5))
log_outputs = np.log(outputs)

for indicator in outputs.columns:
    left_ax.plot(
        seir_model.times,
        log_outputs[indicator],
        label=indicator,
    )
left_ax.set_xlim(0., 120.)
left_ax.set_xlabel("Time (days)")
left_ax.set_ylim(0., 12.)
left_ax.set_ylabel("ln {number or cumulative number of infectious individuals}")
left_ax.set_title("Influenza")
left_ax.legend()

image = plt.imread("./figures/fig_4_6_flu.jpg")
right_ax.axis("off")
right_ax.imshow(image, aspect="auto");